In [1]:
import pandas as pd 
import json
from sqlalchemy import create_engine

In [2]:
csv_file = "./Resources/data.csv"
data_df = pd.read_csv(csv_file)
data_df

,number_people,date,timestamp,day_of_week,is_weekend,is_holiday,temperature,is_start_of_semester,is_during_semester,month,hour
0,37,2015-08-14 17:00:11-07:00,61211,4,0,0,71.76,0,0,8,17
1,45,2015-08-14 17:20:14-07:00,62414,4,0,0,71.76,0,0,8,17
2,40,2015-08-14 17:30:15-07:00,63015,4,0,0,71.76,0,0,8,17
3,44,2015-08-14 17:40:16-07:00,63616,4,0,0,71.76,0,0,8,17
4,45,2015-08-14 17:50:17-07:00,64217,4,0,0,71.76,0,0,8,17
...,...,...,...,...,...,...,...,...,...,...,...
62179,23,2017-03-18 18:42:28-07:00,67348,5,1,0,61.07,0,1,3,18
62180,21,2017-03-18 18:52:35-07:00,67955,5,1,0,61.07,0,1,3,18
62181,25,2017-03-18 19:02:40-07:00,68560,5,1,0,56.71,0,1,3,19
62182,18,2017-03-18 19:12:47-07:00,69167,5,1,0,56.71,0,1,3,19


In [3]:
member_file = "./Resources/multiTimeline.csv"
timeline_df = pd.read_csv(member_file, skiprows=1)
timeline_df

,Week,gym membership: (United States)
0,2015-08-16,55
1,2015-08-23,66
2,2015-08-30,52
3,2015-09-06,47
4,2015-09-13,43
...,...,...
78,2017-02-12,70
79,2017-02-19,63
80,2017-02-26,62
81,2017-03-05,61


In [4]:
crowds_df = data_df[['timestamp', 'date', 'number_people', 'day_of_week', 'month','hour']].copy()
crowds_df

,timestamp,date,number_people,day_of_week,month,hour
0,61211,2015-08-14 17:00:11-07:00,37,4,8,17
1,62414,2015-08-14 17:20:14-07:00,45,4,8,17
2,63015,2015-08-14 17:30:15-07:00,40,4,8,17
3,63616,2015-08-14 17:40:16-07:00,44,4,8,17
4,64217,2015-08-14 17:50:17-07:00,45,4,8,17
...,...,...,...,...,...,...
62179,67348,2017-03-18 18:42:28-07:00,23,5,3,18
62180,67955,2017-03-18 18:52:35-07:00,21,5,3,18
62181,68560,2017-03-18 19:02:40-07:00,25,5,3,19
62182,69167,2017-03-18 19:12:47-07:00,18,5,3,19


In [7]:
tl_df = timeline_df[['Week','gym membership: (United States)']]
tl_df

,Week,gym membership: (United States)
0,2015-08-16,55
1,2015-08-23,66
2,2015-08-30,52
3,2015-09-06,47
4,2015-09-13,43
...,...,...
78,2017-02-12,70
79,2017-02-19,63
80,2017-02-26,62
81,2017-03-05,61


In [12]:
times = crowds_df.rename(columns={'timestamp':'id',
                           'number_people':'members',
                           'date':'date',
                           'hour':'hour',
                           'day_of_week':'weekday',
                           'month':'month'})
members = tl_df.rename(columns={'Week':'date',
                                'gym membership: (United States)': 'member_numbers'})

times['date'] = pd.to_datetime(times['date'], utc=True).dt.date
times.drop_duplicates(subset=['date'],inplace=True)
members.drop_duplicates(inplace=True)
times_df = times.reset_index()
times_df.drop('index',axis=1,inplace=True)

In [13]:
times_df.dtypes

id          int64
date       object
members     int64
weekday     int64
month       int64
hour        int64
dtype: object

In [14]:
times_df.head()

,id,date,members,weekday,month,hour
0,61211,2015-08-15,37,4,8,17
1,61215,2015-08-16,26,5,8,17
2,61213,2015-08-17,31,6,8,17
3,61210,2015-08-18,47,0,8,17
4,61213,2015-08-19,45,1,8,17


In [15]:
members.dtypes

date              object
member_numbers     int64
dtype: object

In [16]:
members.head()

,date,member_numbers
0,2015-08-16,55
1,2015-08-23,66
2,2015-08-30,52
3,2015-09-06,47
4,2015-09-13,43


In [17]:
protocol = 'postgresql'
username = 'postgres'
password = 'Bryan&Kim2018'
host = 'localhost'
port = 5432
database_name = 'exercise_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [18]:
engine.table_names()

C:\Users\kls4f\AppData\Local\Temp\ipykernel_22768\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['times', 'members', 'crowd']

In [22]:
times_df.to_sql(name='times',con=engine,if_exists='append',index=False)

540

In [23]:
members.to_sql(name='members', con=engine, if_exists='append',index=False)

83